<a href="https://colab.research.google.com/github/gauthier2170/Projet-Mars-2025/blob/main/Projet_Avril_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

model_name = "gpt2-medium"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # nécessaire pour éviter les erreurs de padding

model = AutoModelForCausalLM.from_pretrained(model_name)
model.eval()
model.to("cuda" if torch.cuda.is_available() else "cpu")

generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)

examples = [
    "What is the capital of France?",
    "What are the three primary colors?",
    "What does DNA stand for?"
]

for instruction in examples:
    prompt = f"Instruction: {instruction}\nRéponse:"
    output = generator(prompt, max_new_tokens=60)[0]["generated_text"]
    print("\n➡️", instruction)
    print(output)


Device set to use cuda:0



➡️ What is the capital of France?
Instruction: What is the capital of France?
Réponse: You are asking the wrong question. The capital of France – which refers to the town of Paris in the south of the country – is the province of the City of Paris. The French government established Paris as its capital in 1789 following the collapse of Napoleon's Spanish empire and by 1808 the capital

➡️ What are the three primary colors?
Instruction: What are the three primary colors?
Réponse: Red, Yellow, and Blue is what I like to call my "yellow" and "blue" colors! It's the shade in the center of the picture. The first is the color yellow, the second is the color red, and the third is the color blue. Now look at the four

➡️ What does DNA stand for?
Instruction: What does DNA stand for?
Réponse: DNA stands for decoded instructions for the protein synthesis, protein degradation and protein delivery. Protein decapitation and breakdown processes, such as adenosine triphosphate (ATP), phosphocreatine,

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

# Charger Alpaca, en gardant seulement 'instruction' et 'output'
dataset = load_dataset("tatsu-lab/alpaca")["train"]
dataset = dataset.remove_columns(["input"])

# Optionnel : filtrer les réponses longues (> 80 mots)
def is_simple(example):
    return len(example["output"].split()) <= 80

dataset = dataset.filter(is_simple)

#dataset = dataset.select(range(3000))  # au lieu de 1000


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2-medium")
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 n'a pas de token de padding à la base


In [ ]:
def tokenize(example):
    prompt = f"Instruction: {example['instruction']}\nRéponse:"
    full_text = prompt + " " + example["output"]
    tokenized = tokenizer(full_text, truncation=True, max_length=128, padding="max_length")
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(tokenize, remove_columns=dataset.column_names)


Map:   0%|          | 0/42671 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForCausalLM

base_model = AutoModelForCausalLM.from_pretrained("gpt2-medium")
base_model.resize_token_embeddings(len(tokenizer))  # Pour intégrer le token de padding


Embedding(50257, 1024)

In [ ]:
from peft import get_peft_model, LoraConfig, TaskType

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["c_attn"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

model = get_peft_model(base_model, lora_config)


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

output_dir = "./gpt2-medium-alpaca-lora"

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    logging_steps=1000,
    learning_rate=3e-4,
    num_train_epochs=2,
    fp16=torch.cuda.is_available(),
    save_strategy="epoch",
    save_total_limit=1,
    report_to=[],
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
)

trainer.train()


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss


KeyboardInterrupt: 

In [ ]:
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('./gpt2-medium-alpaca-lora/tokenizer_config.json',
 './gpt2-medium-alpaca-lora/special_tokens_map.json',
 './gpt2-medium-alpaca-lora/vocab.json',
 './gpt2-medium-alpaca-lora/merges.txt',
 './gpt2-medium-alpaca-lora/added_tokens.json',
 './gpt2-medium-alpaca-lora/tokenizer.json')

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)

instruction = "What is the capital of France?"
prompt = f"Instruction: {instruction}\nRéponse:"
result = pipe(
    prompt,
    max_new_tokens=80,
    temperature=0.7,      # plus conservateur
    top_p=0.9,            # nucleus sampling
    top_k=50,             # coupe les options absurdes
    eos_token_id=tokenizer.eos_token_id
)[0]["generated_text"]

print("\n🧠 Réponse générée :\n")
print(result)


Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJa


🧠 Réponse générée :

Instruction: What is the capital of France?
Réponse: France is the capital of France. The capital of France is Paris. The capital of France is Paris, the capital of France.
The capital of France is Paris. The capital of France is Paris.
The capital of France is Paris. The capital of France is Paris. The capital of France is Paris.
The capital of France is Paris. The capital of France is Paris. The capital


In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# 💬 Instructions de test
eval_instructions = [
    "What is the capital of France?",
    "Give three tips for staying healthy.",
    "Create a list of five different animals",
    "Who wrote 'Romeo and Juliet'?"
]

# 🚀 Paramètres de génération (réglés pour du bon contrôle)
gen_kwargs = dict(
    max_new_tokens=80,
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
    eos_token_id=50256  # Fin de séquence pour GPT2
)

# 📥 Charger modèle de base
model_base = AutoModelForCausalLM.from_pretrained("gpt2-medium")
tokenizer_base = AutoTokenizer.from_pretrained("gpt2-medium")

# 📥 Charger modèle fine-tuné
model_ft = AutoModelForCausalLM.from_pretrained("gpt2-medium")
tokenizer_ft = AutoTokenizer.from_pretrained("gpt2-medium")
model_ft.resize_token_embeddings(len(tokenizer_ft))
model_ft = PeftModel.from_pretrained(model_ft, "./gpt2-medium-alpaca-lora")

# ⚡ Pipelines
pipe_base = pipeline("text-generation", model=model_base, tokenizer=tokenizer_base, device_map="auto")
pipe_ft = pipeline("text-generation", model=model_ft, tokenizer=tokenizer_ft, device_map="auto")

# 🔍 Comparaison
for instr in eval_instructions:
    prompt = f"Instruction: {instr}\nRéponse:"

    base_resp = pipe_base(prompt, **gen_kwargs)[0]["generated_text"]
    ft_resp = pipe_ft(prompt, **gen_kwargs)[0]["generated_text"]

    # 🌟 Affichage propre
    print("\n" + "="*80)
    print(f"📌 Instruction: {instr}")
    print("\n🧠 Réponse GPT2 (base):")
    print(base_resp.replace(prompt, "").strip())

    print("\n🧪 Réponse GPT2 fine-tuné:")
    print(ft_resp.replace(prompt, "").strip())
    print("="*80)


Device set to use cuda:0
Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeo


📌 Instruction: What is the capital of France?

🧠 Réponse GPT2 (base):
The capital of France is Paris.
Question: What is the capital of France?
Réponse: The capital of France is Paris.
Question: What is the capital of France?
Réponse: The capital of France is Paris.
Question: What is the capital of France?
Réponse: The capital of France is Paris.
Question: What

🧪 Réponse GPT2 fine-tuné:
The capital of France is Lyon, France. The capital is located in the north of France, about 60 km from the French border. It is the most populous city in France and has a population of about 8.7 million people. Lyon has a history of being a center of learning, art, and commerce, as well as a center of commerce and industry. The city is located on the French


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



📌 Instruction: Give three tips for staying healthy.

🧠 Réponse GPT2 (base):
I can't believe I'm so wrong. I know, I know. It's not the healthiest thing to say, but it's true.
I've spent my whole life fighting for my body to look perfect. I've spent my whole life trying to look perfect. I've spent my whole life trying to look perfect.
But then I found myself in this place where I was

🧪 Réponse GPT2 fine-tuné:
1. Eat healthy foods.
2. Exercise regularly.
3. Take regular breaks from work.
4. Get enough sleep.
5. Avoid high-calorie foods such as sugary drinks and fast foods.
6. Drink plenty of fluids.
7. Eat plenty of fruits and vegetables.
8. Stay active.
9. Reduce stress and anxiety.
10


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



📌 Instruction: Create a list of five different animals

🧠 Réponse GPT2 (base):
-Create a list of five different animals
-Create a list of five different animals
-Create a list of five different animals
-Create a list of five different animals
-Create a list of five different animals
-Create a list of five different animals
-Create a list of five different animals
-Create a list of five different animals
-Create a list of five different

🧪 Réponse GPT2 fine-tuné:
- Wild boar 
- Sheep 
- Horse 
- Dog 
- Elephant 
- Tiger 
- Tiger 
- Bear 
- Wild boar 
- Sheep 
- Horse 
- Dog 
- Elephant 
- Tiger 
- Tiger 
- Bear 
- Wild boar 
- Sheep 
- Horse


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



📌 Instruction: Who wrote 'Romeo and Juliet'?

🧠 Réponse GPT2 (base):
I'm not sure. I think the one I read was by William Shakespeare.
Proust: The one I read was by Shakespeare.
Réponse: What's the name of the play?
Proust: Romeo and Juliet.
Réponse: What's the name of the play?
Proust: Romeo and Juliet.
Réponse

🧪 Réponse GPT2 fine-tuné:
Shakespeare wrote 'Romeo and Juliet' in 1516. It was the first Shakespeare play to be adapted for the stage and was written in Latin and Latin-English. The play is considered one of the most popular Shakespeare plays and has been performed in over 200 countries around the world. The story revolves around Romeo and Juliet, two lovers who have a falling out. In the play, Romeo falls


In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# 🔁 Charger modèle fine-tuné
base_model = AutoModelForCausalLM.from_pretrained("gpt2-medium")
tokenizer = AutoTokenizer.from_pretrained("gpt2-medium")
base_model.resize_token_embeddings(len(tokenizer))

# Charger les poids LoRA
model = PeftModel.from_pretrained(base_model, "./gpt2-medium-alpaca-lora")

# 🎯 Pipeline avec réglages pour qualité
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto"
)

# 🔧 Paramètres de génération
gen_kwargs = dict(
    max_new_tokens=100,
    temperature=0.7,      # Gère la créativité (0.7 = bon équilibre)
    top_p=0.9,            # Nucleus sampling
    do_sample=True,
    eos_token_id=50256    # Pour forcer l’arrêt en fin de phrase
)

# 🧪 Interface utilisateur
print("🦙 Assistant Fine-tuné Alpaca | Tape 'exit' pour quitter\n")

while True:
    instr = input("📝 Instruction: ")
    if instr.lower() in ["exit", "quit"]:
        break

    prompt = f"Instruction: {instr}\nRéponse:"

    try:
        result = pipe(prompt, **gen_kwargs)[0]["generated_text"]
        print("\n🧠 Réponse générée :")
        print(result.replace(prompt, "").strip())
        print("=" * 80 + "\n")
    except Exception as e:
        print("⚠️ Erreur :", e)


Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJa

🦙 Assistant Fine-tuné Alpaca | Tape 'exit' pour quitter

📝 Instruction: What is the capital of France?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



🧠 Réponse générée :
France is the capital of France, and the capital of France is Paris. The capital of France is Paris, with the city also being the third largest city in France. The capital of France is Paris, with the city also being the third largest city in France. The capital of France is Paris, with the city also being the third largest city in France. The capital of France is Paris, with the city also being the third largest city in France. The capital of France is Paris, with the

📝 Instruction: What is the capital of France?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



🧠 Réponse générée :
The three primary colors are red, green, and blue. Red is the primary color of the Sun, and green is the primary color of the Moon. Blue is the primary color of the Earth, and green is the primary color of the Sun. Yellow is the primary color of the Earth, and red is the primary color of the Moon. Yellow and blue are the three primary colors of the Moon. The third primary color is cyan, which is the color of the Earth's surface. Cyan is

📝 Instruction: Give three tips for staying healthy.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



🧠 Réponse générée :
1. Eat healthy foods, such as fruits, vegetables, whole grains, and lean proteins. 
2. Stay active and get enough sleep. 
3. Exercise regularly to burn fat and improve blood sugar control. 
4. Eat healthy fats, such as nuts, seeds, avocado, and olive oil. 
5. Limit alcohol and tobacco consumption. 
6. Reduce stress and anxiety. 
7. Avoid caffeine and sugar. 
8. Reduce the amount



KeyboardInterrupt: Interrupted by user